In [4]:
import importlib
import dataingestion.DataIngestion
import datastorage.DataStorage
import datavalidation.DataValidation
import datapreparation.DataPreparation
import datatransformationandstorage.DataTransformationAndStorage
import featurestore.FeatureStore
import dataversioning.DataVersioning
import modelbuild.ModelBuild
import os
importlib.reload(dataingestion.DataIngestion)
importlib.reload(datastorage.DataStorage)
importlib.reload(datavalidation.DataValidation)
importlib.reload(datapreparation.DataPreparation)
importlib.reload(datatransformationandstorage.DataTransformationAndStorage)
importlib.reload(featurestore.FeatureStore)
importlib.reload(dataversioning.DataVersioning)
importlib.reload(modelbuild.ModelBuild)


from prefect import task, flow, get_run_logger
from prefect.tasks import Task
from dataingestion.DataIngestion import load_csv, load_api, load_db
from datastorage.DataStorage import save_csv_or_db, save_api
from datavalidation.DataValidation import validate_churn_data
from datapreparation.DataPreparation import preprocess_and_eda
from datatransformationandstorage.DataTransformationAndStorage import transform_and_store
from featurestore.FeatureStore  import create_feature_store, sample_feature_queries
from dataversioning.DataVersioning import save_and_version_both
from modelbuild.ModelBuild import run_training
import sqlite3  
from graphviz import Digraph
import os
Digraph.format = "png"   # ensures inline rendering


# Define dependencies between tasks
dag_dependencies = {
    "ingest_data": ["store_data", "validate_data", "prepare_data"],
    "prepare_data": ["transform_data"],
    "transform_data": ["build_feature_store"],
    "build_feature_store": ["train_model", "version_data"]
}



def draw_dag(dependencies, title="Churn ML Pipeline"):
    dot = Digraph(comment=title, format="png")

    # Add all tasks as nodes
    tasks = set(dependencies.keys()) | {t for deps in dependencies.values() for t in deps}
    for task in tasks:
        dot.node(task, task)

    # Add edges
    for parent, children in dependencies.items():
        for child in children:
            dot.edge(parent, child)

    output_folder = "results"
    os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist
    output_file = os.path.join(output_folder, "orchestor.dot")
    with open(output_file, "w") as f:
        f.write(dot.source)

    return dot
    
@task
def ingest_data():
    logger = get_run_logger()
    csv_url = "https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv"
    logger.info(f"📥 Ingesting data from {csv_url}")
    df_csv = load_csv(csv_url, csv_url)
    logger.info(f"✅ Data ingestion complete. Shape: {df_csv.shape}")
    return df_csv

@task
def store_data(df_csv):
    logger = get_run_logger()
    base_dir = "results/store_data"
    save_csv_or_db(df_csv, base_dir, "csv")
    logger.info(f"✅ Data stored at {base_dir}")
    return base_dir

@task
def validate_data(df_csv):
    logger = get_run_logger()
    base_dir = "results/validate_data_reports"
    issues, metadata = validate_churn_data(df_csv, base_dir, "pdf")
    logger.info(f"🔍 Validation complete. Issues: {len(issues)} Metadata: {metadata}")
    return issues, metadata

@task
def prepare_data(df_csv):
    logger = get_run_logger()
    base_dir = "results/prepared_data"
    df_processed = preprocess_and_eda(df_csv, base_dir)
    logger.info(f"✅ Data preparation complete. Shape: {df_processed.shape}")
    return df_processed

@task
def transform_data(df_processed):
    logger = get_run_logger()
    base_dir = "results/transformation_and_storage"
    df_txfnstr = transform_and_store(df_processed, base_dir, "churn")
    logger.info(f"✅ Data transformation complete. Shape: {df_txfnstr.shape}")
    return df_txfnstr

@task
def build_feature_store(df_txfnstr):
    logger = get_run_logger()
    base_path = "results/featurestore"
    df_feature, conn, db_path = create_feature_store(df_txfnstr, base_path)
    sample_feature_queries(conn, base_path)
    logger.info(f"✅ Feature store created at {base_path}, DB path: {db_path}")
    return df_feature, db_path

@task
def version_data(df_csv, df_feature):
    logger = get_run_logger()
    save_and_version_both(
        df_csv,
        df_feature,
        "results/dataversion/churn_raw.csv",
        "results/dataversion/churn_transformed_v1.csv",
        "churn_raw",
        "Pipeline_runnning_updates"
    )
    logger.info("✅ Data versioning complete.")

@task
def train_model(db_path):
    logger = get_run_logger()
    run_training(db_path)
    logger.info("✅ Model training complete.")


@flow(name="Churn ML Pipeline Orchestration")
def churn_pipeline():
    df_csv = ingest_data()
    store_data(df_csv)
    validate_data(df_csv)
    df_processed = prepare_data(df_csv)
    df_txfnstr = transform_data(df_processed)
    df_feature, db_path = build_feature_store(df_txfnstr)
    version_data(df_csv, df_feature)
    train_model(db_path)
    dot = draw_dag(dag_dependencies)
    print("✅ Pipeline complete!")

# Draw DAG
   





if __name__ == "__main__":
    churn_pipeline()

11:03:27.381 | INFO    | Flow run 'brave-mink' - Beginning flow run 'brave-mink' for flow 'Churn ML Pipeline Orchestration'

11:03:27.402 | INFO    | Task run 'ingest_data-aff' - 📥 Ingesting data from https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv

2025-08-24 11:03:27,404 [INFO] [https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv] CSV data loading from https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv
2025-08-24 11:03:27,404 [INFO] [https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv] CSV data loading from https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv


11:03:27.404 | INFO    | IngestionLoggerResult - [https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv] CSV data loading from https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv

2025-08-24 11:03:34,861 [INFO] [https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv] CSV data loaded with shape (10000, 14)
2025-08-24 11:03:34,861 [INFO] [https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv] CSV data loaded with shape (10000, 14)


...

11:05:02.650 | ERROR   | Task run 'version_data-6c3' - Task run failed with exception: CalledProcessError(1, ['git', 'push', 'origin', 'main']) - No retries configured for this task.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 869, in run_context
    yield self
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 1505, in run_task_sync
    engine.call_task_fn(txn)
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 886, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/utilities/callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/sk/qzp6psz960qdsy0d5vhbz84w0000gn/T/ipykernel_9308/507818884.py", line 121, in version_data
    save_and_version_both(
  File "/Users/dhairyas87/DMML_Machine_Learning_Pipeline/dataversioning/DataVersioning.py", line 77, in save_and_version_both
    subprocess.run(["git", "push", remote, branch], check=True)
  File "/opt/anaconda3/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['git', 'push', 'origin', 'main']' returned non-zero exit status 1.

11:05:02.659 | ERROR   | Task run 'version_data-6c3' - Finished in state Failed("Task run encountered an exception CalledProcessError: Command '['git', 'push', 'origin', 'main']' returned non-zero exit status 1.")

11:05:02.660 | ERROR   | Flow run 'brave-mink' - Encountered exception during execution: CalledProcessError(1, ['git', 'push', 'origin', 'main'])
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/flow_engine.py", line 782, in run_context
    yield self
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/flow_engine.py", line 1397, in run_flow_sync
    engine.call_flow_fn()
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/flow_engine.py", line 802, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/utilities/callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/sk/qzp6psz960qdsy0d5vhbz84w0000gn/T/ipykernel_9308/507818884.py", line 146, in churn_pipeline
    version_data(df_csv, df_feature)
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/tasks.py", line 1139, in __call__
    return run_task(
           ^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 1732, in run_task
    return run_task_sync(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 1507, in run_task_sync
    return engine.state if return_type == "state" else engine.result()
                                                       ^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 494, in result
    raise self._raised
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 869, in run_context
    yield self
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 1505, in run_task_sync
    engine.call_task_fn(txn)
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/task_engine.py", line 886, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/prefect/utilities/callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/sk/qzp6psz960qdsy0d5vhbz84w0000gn/T/ipykernel_9308/507818884.py", line 121, in version_data
    save_and_version_both(
  File "/Users/dhairyas87/DMML_Machine_Learning_Pipeline/dataversioning/DataVersioning.py", line 77, in save_and_version_both
    subprocess.run(["git", "push", remote, branch], check=True)
  File "/opt/anaconda3/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['git', 'push', 'origin', 'main']' returned non-zero exit status 1.

11:05:02.682 | INFO    | Flow run 'brave-mink' - Finished in state Failed("Flow run encountered an exception: CalledProcessError: Command '['git', 'push', 'origin', 'main']' returned non-zero exit status 1.")

CalledProcessError: Command '['git', 'push', 'origin', 'main']' returned non-zero exit status 1.